In [25]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [26]:
#data
df=pd.read_csv('C:\Kate\Python\Authorship Attribution\data\AllTweets.csv')
author1='DonaldTrump'
author2='BarackObama'
df_kk=df.loc[(df['author'] == author1)]
df_hc=df.loc[(df['author'] == author2)]
df=df_kk.append(df_hc,ignore_index=True)
len(df)

24112

In [27]:
import random
#2000 random sample rows for KK
rows = random.sample(df_kk.index, 2000)
df_kk = df_kk.ix[rows]
#2000 random sample rows for HC
rows = random.sample(df_hc.index, 2000)
df_hc = df_hc.ix[rows]
#join back together
df=df_kk.append(df_hc,ignore_index=True)
len(df)

4000

In [28]:
#data pre-processing
df.drop(df[df.retweet==True].index, inplace=True)
df['num_of_words'] = df["text"].str.split().apply(len)
df.drop(df[df.num_of_words<4].index, inplace=True)
df["text"].replace(r"http\S+", "URL", regex=True,inplace=True)
df["text"].replace(r"@\S+", "REF", regex=True ,inplace=True)
df["text"].replace(r"(\d{1,2})[/.-](\d{1,2})[/.-](\d{2,4})+", "DATE", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"(\d{1,2})[/:](\d{2})?(am|pm)+", "TIME", regex=True,inplace=True)
df["text"].replace(r"\d+", "NUM", regex=True,inplace=True)
len(df)

3506

In [29]:
#http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
pos_code_map={'CC':'A','CD':'B','DT':'C','EX':'D','FW':'E','IN':'F','JJ':'G','JJR':'H','JJS':'I','LS':'J','MD':'K','NN':'L','NNS':'M',
'NNP':'N','NNPS':'O','PDT':'P','POS':'Q','PRP':'R','PRP$':'S','RB':'T','RBR':'U','RBS':'V','RP':'W','SYM':'X','TO':'Y','UH':'Z',
'VB':'1','VBD':'2','VBG':'3','VBN':'4','VBP':'5','VBZ':'6','WDT':'7','WP':'8','WP$':'9','WRB':'@'}
code_pos_map={v: k for k, v in pos_code_map.iteritems()}
#Python 3 inv_map = {v: k for k, v in my_map.items()}

In [30]:
def convert(tag):
    try:
        code=pos_code_map[tag]
    except:
        code='?'
    return code
def inv_convert(code):
    try:
        tag=code_pos_map[code]
    except:
        tag='?'
    return tag

In [31]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, word_tokenize
def pos_tags(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    return "".join(convert(tag) for (word, tag) in nltk.pos_tag(text_processed))
def text_pos_inv_convert(text):
    return "-".join(inv_convert(c.upper()) for c in text)

In [32]:
df['text_pos']=df.apply(lambda x: pos_tags(x['text']), axis=1)

In [33]:
df.ix[:,['author','text','text_pos']].head()

,author,text,text_pos
2,DonaldTrump,Small crowds at REF today in Atlanta. People w...,GLFNLFNN2TGFNCGLAL864RLAL
3,DonaldTrump,“Donald Trump dedicates second Scottish golf c...,LNN6GGLLY1LNNFN
5,DonaldTrump,Looking forward to tonight's Ayrshire Chamber ...,3TY1GNNFNNNNN
6,DonaldTrump,Failing REF lost all credibility. Not only was...,3N2CLTT2R2NRK14FNFNN
7,DonaldTrump,“In every battle there comes a time when both ...,LFCLT6CL@CM5R1


In [34]:
df_features=pd.DataFrame()

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
for a in df.author.unique():
    v = CountVectorizer(analyzer='char',ngram_range=(7, 7))
    ngrams = v.fit_transform(df[df['author'] == a]['text_pos'])
    df_t=pd.DataFrame(
    {'Feature': v.get_feature_names(),
     'Count': list(ngrams.sum(axis=0).flat),
     'Author': a
    })
    #
    df_features=df_features.append(df_t,ignore_index=True)

In [36]:
df_features['Feature_POS']=df_features.apply(lambda x: text_pos_inv_convert(x['Feature']), axis=1)

In [37]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author2].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

,Author,Count,Feature,Feature_POS
39305,BarackObama,8,r2lfny1,PRP-VBD-NN-IN-NNP-TO-VB
34503,BarackObama,8,lynllll,NN-TO-NNP-NN-NN-NN-NN
27116,BarackObama,7,fcgllll,IN-DT-JJ-NN-NN-NN-NN
27364,BarackObama,7,fclllll,IN-DT-NN-NN-NN-NN-NN
25667,BarackObama,6,clfclnn,DT-NN-IN-DT-NN-NNP-NNP


In [38]:
df_features[~df_features.Feature.isin(df_features[df_features['Author'] != author1].Feature)].sort_values('Count', ascending=False).ix[:,['Author','Count','Feature','Feature_POS']].head()

,Author,Count,Feature,Feature_POS
13933,DonaldTrump,7,nfnnfnn,NNP-IN-NNP-NNP-IN-NNP-NNP
15526,DonaldTrump,7,nnnnnnf,NNP-NNP-NNP-NNP-NNP-NNP-IN
15041,DonaldTrump,6,nnfnnfn,NNP-NNP-IN-NNP-NNP-IN-NNP
15370,DonaldTrump,6,nnnfnnf,NNP-NNP-NNP-IN-NNP-NNP-IN
15411,DonaldTrump,5,nnnlfnn,NNP-NNP-NNP-NN-IN-NNP-NNP


In [39]:
from sklearn.cross_validation import train_test_split
twt_train, twt_test, author_train, author_test = train_test_split(df.ix[:,['text','text_pos']], df['author'], test_size=0.4, random_state=42)

In [40]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
def text_process(text):
    """
    Takes in a string of text, then performs the following:
    1. Tokenizes and removes punctuation
    3. Stems
    4. Returns a list of the cleaned text
    """

    # tokenizing
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    
    # steming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    

    return text_processed

In [41]:
ScoreSummaryByModelParams = list()

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

In [43]:
from sklearn.base import BaseEstimator, TransformerMixin
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [44]:
class TextAndTextCodedExtractor(BaseEstimator, TransformerMixin):
    """Extract the text & text_pos from a tweet in a single pass.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, tweets):
        features=tweets.ix[:,['text_pos','text']].to_records(index=False)

        return features

In [45]:
def ModelParamsEvaluation (f_union,model,params,comment):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    grid_search = GridSearchCV(estimator=pipeline, param_grid=params, verbose=1, cv=10)
    grid_search.fit(twt_train, author_train)
    #best score
    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        ScoreSummaryByModelParams.append([comment,grid_search.best_score_,"\t%s: %r" % (param_name, best_parameters[param_name])])    
 

In [46]:
f1_union=FeatureUnion(
        transformer_list=[
              # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char')),
            ])),               

        ],
    )

In [47]:
from sklearn.naive_bayes import BernoulliNB
p = {
    'union__char__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__char__tfidf__ngram_range': ((2, 2), (3, 3)), 
    'union__char__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char')

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    8.6s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   38.7s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:  1.5min
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  2.5min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  3.7min


Best score: 0.941
Best parameters set:
	clf__alpha: 0.01
	union__char__tfidf__max_df: 0.5
	union__char__tfidf__max_features: 5000
	union__char__tfidf__ngram_range: (3, 3)


[Parallel(n_jobs=1)]: Done 1440 out of 1440 | elapsed:  4.3min finished


In [48]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word')),
            ])),              

        ],
    )

In [49]:
p = {
    'union__word__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__word__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3),(4,4),(5,5)), 
    'union__word__tfidf__max_features': (None, 5000, 10000, 50000),
    'union__word__tfidf__stop_words': (None, 'english'),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, word')

Fitting 10 folds for each of 720 candidates, totalling 7200 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    5.8s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   25.7s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   58.4s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  1.7min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  2.7min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  3.9min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  5.4min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed:  7.0min
[Parallel(n_jobs=1)]: Done 4049 tasks       | elapsed:  8.9min
[Parallel(n_jobs=1)]: Done 4999 tasks       | elapsed: 11.0min
[Parallel(n_jobs=1)]: Done 6049 tasks       | elapsed: 13.3min
[Parallel(n_jobs=1)]: Done 7199 tasks       | elapsed: 15.9min
[Parallel(n_jobs=1)]: Done 7200 out of 7200 | elapsed: 15.9min finished


Best score: 0.924
Best parameters set:
	clf__alpha: 0.1
	union__word__tfidf__max_df: 0.75
	union__word__tfidf__max_features: None
	union__word__tfidf__ngram_range: (1, 1)
	union__word__tfidf__stop_words: None


In [50]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling word features from the text
            ('text', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process)),
            ])),              

        ],
    )

In [51]:
p = {
    'union__text__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__text__tfidf__ngram_range': ((1, 1),(2, 2), (3, 3),(4,4),(5,5)), 
    'union__text__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, stemmed words, no stop words')

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   22.7s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:  1.5min
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:  3.5min
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  6.2min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  9.6min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed: 13.9min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed: 19.0min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed: 24.8min
[Parallel(n_jobs=1)]: Done 3600 out of 3600 | elapsed: 27.9min finished


Best score: 0.924
Best parameters set:
	clf__alpha: 1
	union__text__tfidf__max_df: 0.75
	union__text__tfidf__max_features: None
	union__text__tfidf__ngram_range: (1, 1)


In [52]:
f1_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling pos tag features  from the text_pos
            ('text_pos', Pipeline([
            ('selector', ItemSelector(key='text_pos')),
            ('tfidf',    TfidfVectorizer(analyzer='char')),
            ])),                  

        ],
    )

In [53]:
p = {
    'union__text_pos__tfidf__max_df': (0.5, 0.75, 1.0),
    'union__text_pos__tfidf__ngram_range': ((3, 3), (4, 4),(5,5),(6,6),(7,7)), 
    'union__text_pos__tfidf__max_features': (None, 5000, 10000, 50000),
    'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f1_union,BernoulliNB(),p,'Bernoulli Naive Bayes, POS tags')

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    4.9s
[Parallel(n_jobs=1)]: Done 199 tasks       | elapsed:   20.0s
[Parallel(n_jobs=1)]: Done 449 tasks       | elapsed:   44.8s
[Parallel(n_jobs=1)]: Done 799 tasks       | elapsed:  1.3min
[Parallel(n_jobs=1)]: Done 1249 tasks       | elapsed:  2.1min
[Parallel(n_jobs=1)]: Done 1799 tasks       | elapsed:  3.0min
[Parallel(n_jobs=1)]: Done 2449 tasks       | elapsed:  4.1min
[Parallel(n_jobs=1)]: Done 3199 tasks       | elapsed:  5.4min


Best score: 0.689
Best parameters set:
	clf__alpha: 1
	union__text_pos__tfidf__max_df: 0.5
	union__text_pos__tfidf__max_features: None
	union__text_pos__tfidf__ngram_range: (3, 3)


[Parallel(n_jobs=1)]: Done 3600 out of 3600 | elapsed:  6.0min finished


In [54]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

,Method,BestScore,BestParameter
0,"Bernoulli Naive Bayes, char",0.940561,\tclf__alpha: 0.01
1,"Bernoulli Naive Bayes, char",0.940561,\tunion__char__tfidf__max_df: 0.5
2,"Bernoulli Naive Bayes, char",0.940561,\tunion__char__tfidf__max_features: 5000
3,"Bernoulli Naive Bayes, char",0.940561,"\tunion__char__tfidf__ngram_range: (3, 3)"
12,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.924394,"\tunion__text__tfidf__ngram_range: (1, 1)"
11,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.924394,\tunion__text__tfidf__max_features: None
10,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.924394,\tunion__text__tfidf__max_df: 0.75
9,"Bernoulli Naive Bayes, stemmed words, no stop ...",0.924394,\tclf__alpha: 1
8,"Bernoulli Naive Bayes, word",0.923918,\tunion__word__tfidf__stop_words: None
7,"Bernoulli Naive Bayes, word",0.923918,"\tunion__word__tfidf__ngram_range: (1, 1)"


In [55]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [56]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f2_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   13.3s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   16.3s finished


Best score: 0.947
Best parameters set:
	clf__alpha: 0.01


In [57]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [58]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f2_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + stemmed word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   28.3s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   34.7s finished


Best score: 0.951
Best parameters set:
	clf__alpha: 0.001


In [59]:
f3_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),    
            # Pipeline for pulling word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),        

        ],

    )

In [60]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + stemmed word')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   31.4s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   38.6s finished


Best score: 0.950
Best parameters set:
	clf__alpha: 0.001


In [61]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [62]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + stemmed word + POS tags')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   30.8s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   37.7s finished


Best score: 0.943
Best parameters set:
	clf__alpha: 0.1


In [63]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [64]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}
ModelParamsEvaluation(f3_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + POS tags')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   16.0s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   19.5s finished


Best score: 0.944
Best parameters set:
	clf__alpha: 0.1


In [65]:
f4_union=FeatureUnion(
        transformer_list=[

            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
             # Pipeline for pulling word features after word_processing from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [66]:
p = {'clf__alpha': (1,0.1,0.01,0.001,0.0001,0)}

ModelParamsEvaluation(f4_union,BernoulliNB(),p,'Bernoulli Naive Bayes, char + word + stemmed word + POS tag')

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:   34.0s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   41.6s finished


Best score: 0.947
Best parameters set:
	clf__alpha: 0.01


In [67]:
df_ScoreSummaryByModelParams=DataFrame(ScoreSummaryByModelParams,columns=['Method','BestScore','BestParameter'])
df_ScoreSummaryByModelParams.sort_values(['BestScore'],ascending=False,inplace=True)
df_ScoreSummaryByModelParams

,Method,BestScore,BestParameter
18,"Bernoulli Naive Bayes, char + stemmed word",0.951022,\tclf__alpha: 0.001
19,"Bernoulli Naive Bayes, char + word + stemmed word",0.949596,\tclf__alpha: 0.001
22,"Bernoulli Naive Bayes, char + word + stemmed w...",0.946743,\tclf__alpha: 0.01
17,"Bernoulli Naive Bayes, char + word",0.946743,\tclf__alpha: 0.01
21,"Bernoulli Naive Bayes, char + word + POS tags",0.943890,\tclf__alpha: 0.1
20,"Bernoulli Naive Bayes, char + stemmed word + P...",0.943414,\tclf__alpha: 0.1
1,"Bernoulli Naive Bayes, char",0.940561,\tunion__char__tfidf__max_df: 0.5
0,"Bernoulli Naive Bayes, char",0.940561,\tclf__alpha: 0.01
3,"Bernoulli Naive Bayes, char",0.940561,"\tunion__char__tfidf__ngram_range: (3, 3)"
2,"Bernoulli Naive Bayes, char",0.940561,\tunion__char__tfidf__max_features: 5000


In [87]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc,precision_score, accuracy_score, recall_score, f1_score
from scipy import interp

In [88]:
ScoreSummaryByVector = list()

In [89]:
def PredictionEvaluation(author_test_b,author_predictions_b,comment):
    Precision=precision_score(author_test_b,author_predictions_b)
    print ('Precision: %0.3f' % (Precision))
    Accuracy=accuracy_score(author_test_b,author_predictions_b)
    print ('Accuracy: %0.3f' % (Accuracy))
    Recall=recall_score(author_test_b,author_predictions_b)
    print ('Recall: %0.3f' % (Recall))
    F1=f1_score(author_test_b,author_predictions_b)
    print ('F1: %0.3f' % (F1))
    print ('Confussion matrix:')
    print (confusion_matrix(author_test_b,author_predictions_b))
    ROC_AUC=roc_auc_score(author_test_b,author_predictions_b)
    print ('ROC-AUC: %0.3f' % (ROC_AUC))
    ScoreSummaryByVector.append([Precision,Accuracy,Recall,F1,ROC_AUC,comment])

In [90]:
def ModelRun (f_union,model):
    pipeline = Pipeline([
    # Extract the text & text_coded
    ('textandtextcoded', TextAndTextCodedExtractor()),

    # Use FeatureUnion to combine the features from text and text_coded
    ('union', f_union, ),

    # Use a  classifier on the combined features
    ('clf', model),
    ])
    pipeline.fit(twt_train, author_train)
    author_predicted = pipeline.predict(twt_test)
    
    feature_names=list()
    for p in (pipeline.get_params()['union'].transformer_list):
        fn=(p[0],pipeline.get_params()['union'].get_params()[p[0]].get_params()['tfidf'].get_feature_names())
        feature_names.append(fn)
    df_fn=pd.DataFrame()
    for fn in feature_names:
        df_fn= df_fn.append(pd.DataFrame(
        {'FeatureType': fn[0],
         'Feature': fn[1]
        }),
        ignore_index=True)    
    
    from sklearn.preprocessing import LabelBinarizer
    lb = LabelBinarizer()
    author_test_b = lb.fit_transform(author_test.values)
    author_predicted_b  = lb.fit_transform(author_predicted)
    return (df_fn,pipeline.get_params()['clf'],author_predicted,author_predicted_b, author_test_b)

In [91]:
def most_informative_feature_for_binary_classification(feature_names, classifier):
    class_labels = classifier.classes_

    topnvalues_class0 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values))
    topnvalues_class1 = sorted(zip(classifier.coef_[0], feature_names['Feature'].values, feature_names['FeatureType'].values), reverse=True)

    topn_df_class0=pd.DataFrame(topnvalues_class0, columns=['Coef','Feature','FeatureType'])
    topn_df_class0['Author']=class_labels[0]
    
    topn_df_class1=pd.DataFrame(topnvalues_class1, columns=['Coef','Feature','FeatureType'])
    topn_df_class1['Author']=class_labels[1]    
    
    topn_df=topn_df_class0.append(topn_df_class1)
    
        
    return topn_df

In [92]:
f2_union=FeatureUnion(
        transformer_list=[
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
            # Pipeline for pulling stememd word features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.75,max_features=None)),
            ])),        

        ],

    )

In [93]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f2_union,BernoulliNB(alpha=0.0001))

In [94]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word')

Precision: 0.943
Accuracy: 0.931
Recall: 0.907
F1: 0.925
Confussion matrix:
[[710  36]
 [ 61 596]]
ROC-AUC: 0.929


In [95]:
f3_union=FeatureUnion(
        transformer_list=[
             # Pipeline for pulling word stemmed features from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.75,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded with POS tags
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [96]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f3_union,BernoulliNB(alpha=0.0001))

In [97]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+stemmed word+POS tag')

Precision: 0.922
Accuracy: 0.923
Recall: 0.908
F1: 0.915
Confussion matrix:
[[712  49]
 [ 59 583]]
ROC-AUC: 0.922


In [98]:
f4_union=FeatureUnion(
        transformer_list=[

            # Pipeline for pulling word features from the text
            ('word', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',    TfidfVectorizer(analyzer='word',stop_words=None,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
             # Pipeline for pulling word features after word_processing from the text
            ('text', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='word',tokenizer= text_process,ngram_range=(1, 1),max_df=0.5,max_features=None)),
            ])),
                    
            # Pipeline for pulling char features  from the text
            ('char', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf',     TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=5000)),
            ])),
                    
            # Pipeline for pulling flexible pattern features  from the text_coded
            ('text_pos', Pipeline([
                ('selector', ItemSelector(key='text_pos')),
                ('tfidf',    TfidfVectorizer(analyzer='char',ngram_range=(3, 3),max_df=0.5,max_features=None)),
            ])),                  

        ],

    )

In [99]:
(feature_names,clf,author_predicted,author_predicted_b, author_test_b)=ModelRun(f4_union,BernoulliNB(alpha=0.0001))

In [100]:
PredictionEvaluation(author_predicted_b, author_test_b,'char+word+stemmed word+POS tag')

Precision: 0.915
Accuracy: 0.924
Recall: 0.916
F1: 0.915
Confussion matrix:
[[718  54]
 [ 53 578]]
ROC-AUC: 0.923


In [101]:
df_ScoreSummaryByVector=DataFrame(ScoreSummaryByVector,columns=['Precision','Accuracy','Recall','F1','ROC-AUC','Vector'])
df_ScoreSummaryByVector.sort_values(['F1'],ascending=False,inplace=True)
df_ScoreSummaryByVector

,Precision,Accuracy,Recall,F1,ROC-AUC,Vector
0,0.943038,0.930862,0.907154,0.924748,0.929448,char+stemmed word
2,0.914557,0.923735,0.916006,0.915281,0.923029,char+word+stemmed word+POS tag
1,0.922468,0.923022,0.908100,0.915228,0.921855,char+stemmed word+POS tag


In [102]:
TopFeatures_df=most_informative_feature_for_binary_classification(feature_names, clf)

In [103]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author2) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

,Author,CoefChar,Char,CoefWord,Word,CoefText,Text,CoefTextPOS,TextPOS
0,BarackObama,-16.023785,"""p",-16.023785,aanumqtlnumonum,-16.023785,aanumqtlnumonum,-16.023785,VB-VB-VB
1,BarackObama,-16.023785,#b,-16.023785,abetterbargain,-16.023785,abetterbargain,-16.023785,VB-VB-WDT
2,BarackObama,-16.023785,#e,-16.023785,abetterbargainurl,-16.023785,abetterbargainurl,-16.023785,VB-VB-IN
3,BarackObama,-16.023785,#h,-16.023785,ability,-16.023785,abil,-16.023785,VB-VB-RP
4,BarackObama,-16.023785,#j,-16.023785,abroad,-16.023785,abroad,-16.023785,VB-VBD-DT
5,BarackObama,-16.023785,#p,-16.023785,absence,-16.023785,absenc,-16.023785,VB-VBG-VBZ
6,BarackObama,-16.023785,#u,-16.023785,abuse,-16.023785,abynumwkxj,-16.023785,VB-VBG-NN
7,BarackObama,-16.023785,(v,-16.023785,abynumwkxj,-16.023785,aca,-16.023785,VB-VBG-NNPS
8,BarackObama,-16.023785,-b,-16.023785,aca,-16.023785,academi,-16.023785,VB-VBG-PRP$
9,BarackObama,-16.023785,ct,-16.023785,academy,-16.023785,acawork,-16.023785,VB-VBG-RB


In [104]:
df1=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='char')),['Author','Coef','Feature']].head(10)
df1.rename(columns={'Coef':'CoefChar','Feature':'Char'}, inplace=True)
df1.reset_index(inplace=True)
df2=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='word')),['Coef','Feature']].head(10)
df2.rename(columns={'Coef':'CoefWord','Feature':'Word'}, inplace=True)
df2.reset_index(inplace=True)
df3=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text')),['Coef','Feature']].head(10)
df3.rename(columns={'Coef':'CoefText','Feature':'Text'}, inplace=True)
df3.reset_index(inplace=True)
df4=TopFeatures_df.loc[((TopFeatures_df['Author']==author1) & (TopFeatures_df['FeatureType']=='text_pos')),['Coef','Feature']].head(10)
df4.rename(columns={'Coef':'CoefTextPOS','Feature':'TextPOS'}, inplace=True)
df4['TextPOS']=df4.apply(lambda x: text_pos_inv_convert(x['TextPOS']), axis=1)
df4.reset_index(inplace=True)
df_kk_top_features = pd.concat([df1,df2,df3,df4],axis=1)
df_kk_top_features.drop('index', axis=1, inplace=True)
df_kk_top_features

,Author,CoefChar,Char,CoefWord,Word,CoefText,Text,CoefTextPOS,TextPOS
0,DonaldTrump,-0.684394,ing,-0.987444,to,-0.987444,to,-1.295992,NNP-NNP-NNP
1,DonaldTrump,-0.754321,ng,-0.993362,ref,-0.993362,ref,-1.582336,IN-NNP-NNP
2,DonaldTrump,-0.785166,re,-1.189427,is,-1.167998,a,-1.671781,DT-JJ-NN
3,DonaldTrump,-0.955511,to,-1.384099,and,-1.189427,is,-1.908169,NN-IN-NNP
4,DonaldTrump,-0.955511,ref,-1.419817,of,-1.384099,and,-1.923095,IN-DT-NN
5,DonaldTrump,-0.996333,is,-1.555949,in,-1.419817,of,-2.068512,DT-NN-IN
6,DonaldTrump,-1.076872,ef,-1.648658,url,-1.555949,in,-2.131313,NNP-NNP-IN
7,DonaldTrump,-1.080103,an,-1.719694,num,-1.582336,i,-2.188471,PRP-MD-VB
8,DonaldTrump,-1.113001,at,-1.744540,on,-1.648658,url,-2.208274,NNP-IN-NNP
9,DonaldTrump,-1.129865,be,-1.829838,for,-1.683546,be,-2.280844,JJ-NN-IN


In [105]:
author_predicted=pd.DataFrame(author_predicted,columns=['predicted'])
df_wrong_result = pd.concat([twt_test.reset_index(),author_test.reset_index(),author_predicted], axis=1)
df_wrong_result.drop('index', axis=1, inplace=True)
df_wrong_result.drop('text_pos', axis=1, inplace=True)
df_wrong_result=df_wrong_result[df_wrong_result['author']<>df_wrong_result['predicted']]
df_wrong_result.head(10)

,text,author,predicted
15,California Republican reps on Twitter: REF (CA...,BarackObama,DonaldTrump
17,Our trade deficit just jumped in May to “the s...,DonaldTrump,BarackObama
31,Thank you for your support at this mornings To...,DonaldTrump,BarackObama
42,Alison Grimes supports harsh restrictions to k...,DonaldTrump,BarackObama
97,"In any business venture, remember that brandin...",DonaldTrump,BarackObama
116,"House GOP wants to cut Medicare, Obama took $N...",DonaldTrump,BarackObama
127,We should expand oil production in America eve...,BarackObama,DonaldTrump
139,I believe in #AmericaFirst and that means FAMI...,DonaldTrump,BarackObama
140,Ron Paul is right that we are wasting trillion...,DonaldTrump,BarackObama
144,"Preparing for our big ""Walk for Change"" nation...",BarackObama,DonaldTrump
